## Fix the eeg.json and channels.tsv files generated by ESS.

This notebook replaces the eeg.json and channels.tsv files generated by ESS with generated files
for the BCIT Calibration Driving data.


In [1]:
import os
import json
from hed.tools import BidsFileDictionary, HedLogger
from hed.util import get_file_list, get_new_dataframe, get_value_dict

# Variables to set for the specific dataset
bids_root_path = 's:/bcit/CalibrationDrivingWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
log_file_name = 'bcit_calibration_driving_07_fix_channels_log.json'
sampling_rate_file = os.path.realpath(os.path.join(bids_root_path, 'code/samplingRates.tsv'))

# Construct the list of .event files
eeg_files = get_file_list(bids_root_path, extensions=[".set"], name_suffix="_eeg", exclude_dirs=exclude_dirs)
eeg_dict = BidsFileDictionary("EEG event files", eeg_files, entities=entities)
sampling_dict = get_value_dict(sampling_rate_file)
channel_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_channels", exclude_dirs=exclude_dirs)

desc_dict = {'LN': 'Lane deviation from center line in meters.',
             'ANG': 'Steering wheel deviation angle in degrees.',
             'SP': 'Vehicle speed in mph.',
             'SD': 'Vehicle speed deviation.'}

json_dict = {
    "TaskName": "Drive",
    "TaskDescription": "Short simulated driving (steering only) with perturbations in a visually sparse environment.",
    "EEGReference": "CMS",
    "SamplingFrequency": 1024.0,
    "RecordingType": "continuous",
    "PowerLineFrequency": 60,
    "SoftwareFilters": "n/a",
    "CapManufacturer": "BioSemi ActiveTwo biopotential measurement system",
    "EEGChannelCount": 64,
    "ECGChannelCount": 0,
    "EMGChannelCount": 0,
    "EOGChannelCount": 0,
    "MiscChannelCount": 6,
    "EEGPlacementScheme": "Biosemi"
}
# Set up the logger and perform  the operations to combine the two versions of the event files
logger = HedLogger()
for key, file in eeg_dict.iter_files():
    filename = eeg_dict.get_file_path(key)
    basename = os.path.basename(filename)
    sampling_rate = float(sampling_dict[basename[:-8]])
    logger.add(key, f"Looked up sampling rate of {sampling_rate}")

    channel_file = filename[:-7] + "channels.tsv"
    logger.add(key, f"Reading the {os.path.basename(channel_file)} channel file")
    df_channels = get_new_dataframe(channel_file)

    channel_mask = df_channels['name'].map(str).isin(['LN', 'ANG', 'SP', 'SD'])
    if sum(channel_mask) == 0:
        continue
    names = df_channels.loc[channel_mask, 'name']
    df_channels.loc[channel_mask, 'type'] = 'OTHER'
    df_channels.loc[channel_mask, 'units'] = 'n/a'
    df_channels.loc[channel_mask, 'sampling_frequency'] = sampling_rate
    for index, name in names.iteritems():
        df_channels.loc[index, 'description'] = desc_dict[name]
        logger.add(key, f"Replacing {name} row with {str([name, 'OTHER', 'n/a', sampling_rate, desc_dict[name]])}")
    eeg_type_mask = df_channels['type'].map(str).isin(['EEG'])
    json_dict["EEGChannelCount"] = sum(eeg_type_mask)
    eog_type_mask = df_channels['type'].map(str).isin(['VEOG', 'HEOG'])
    json_dict["EOGChannelCount"] = sum(eog_type_mask)
    misc_type_mask = df_channels['type'].map(str).isin(['MISC', 'OTHER'])
    json_dict["MiscChannelCount"] = sum(misc_type_mask)
    json_path = filename[:-3] + "json"
    with open(json_path, 'w') as fp:
        json.dump(json_dict, fp, indent=4)
    logger.add(key, f"Saving eeg.json to {os.path.basename(json_path)}")
    df_channels.to_csv(channel_file, sep='\t', index=False)
    logger.add(key, f"Saved as {os.path.basename(channel_file)}")

# Output and save the log
logger.print_log()
save_path = os.path.join(bids_root_path, 'code')
logger.save_log(save_path, log_name=log_file_name)

# Output errors
print("\nERROR Summary:")
logger.print_log(level="ERROR")

sub-01_ses-01_run-1:
	[ Looked up sampling rate of 2048.0]
	[ Reading the sub-01_ses-01_task-Drive_run-1_channels.tsv channel file]
	[ Replacing LN row with ['LN', 'OTHER', 'n/a', 2048.0, 'Lane deviation from center line in meters.']]
	[ Replacing ANG row with ['ANG', 'OTHER', 'n/a', 2048.0, 'Steering wheel deviation angle in degrees.']]
	[ Replacing SP row with ['SP', 'OTHER', 'n/a', 2048.0, 'Vehicle speed in mph.']]
	[ Replacing SD row with ['SD', 'OTHER', 'n/a', 2048.0, 'Vehicle speed deviation.']]
	[ Saving eeg.json to sub-01_ses-01_task-Drive_run-1_eeg.json]
	[ Saved as sub-01_ses-01_task-Drive_run-1_channels.tsv]
sub-02_ses-01_run-1:
	[ Looked up sampling rate of 2048.0]
	[ Reading the sub-02_ses-01_task-Drive_run-1_channels.tsv channel file]
	[ Replacing LN row with ['LN', 'OTHER', 'n/a', 2048.0, 'Lane deviation from center line in meters.']]
	[ Replacing ANG row with ['ANG', 'OTHER', 'n/a', 2048.0, 'Steering wheel deviation angle in degrees.']]
	[ Replacing SP row with ['SP', '